In [1]:
import numpy as np
import tensorflow as tf

import  tensorflow_datasets as tfds 

In [2]:
mnist_dataset,mnist_info = tfds.load(name="mnist",as_supervised=True,with_info=True)

In [3]:
mnist_train, mnist_test = mnist_dataset["train"], mnist_dataset["test"]

num_validation = 0.1 * mnist_info.splits["train"].num_examples
num_validation = tf.cast(num_validation ,tf.int64)

num_test  = mnist_info.splits["test"].num_examples
num_test = tf.cast(num_test ,tf.int64)


def scale(image, label):
    image = tf.cast(image,tf.float32)
    image /= 255.
    return image,label


scale_train = mnist_train.map(scale)

test_data = mnist_test.map(scale)

BUFFER_SIZE = 10000

shuffle_train  = scale_train.shuffle(BUFFER_SIZE)

validation_data = shuffle_train.take(num_validation)
train_data = shuffle_train.skip(num_validation)

BATCH_SIZE = 100

train_data = train_data.batch(BATCH_SIZE)
validation_data = validation_data.batch(num_validation)
test_data = test_data.batch(num_test)

validatio_inputs, validation_targets = next(iter(validation_data))

In [4]:
input_size = 784

output_size = 10

hidden_size = 275

early_stopping = tf.keras.callbacks.EarlyStopping(patience=2)

model = tf.keras.Sequential([
    tf.keras.layers.Flatten(input_shape=(28,28,1)),
    tf.keras.layers.Dense(hidden_size,activation='relu'),
    tf.keras.layers.Dense(hidden_size,activation='relu'),
    tf.keras.layers.Dense(output_size,activation='softmax'),
    
])


In [6]:

model.compile(optimizer="adam", loss="sparse_categorical_crossentropy", metrics=["accuracy"])

In [ ]:
num_epochs = 5

model.fit(train_data, epochs=num_epochs, validation_data=(validatio_inputs, validation_targets),verbose=2,callbacks=[early_stopping])

Epoch 1/5
540/540 - 4s - loss: 0.2541 - accuracy: 0.9264 - val_loss: 0.1219 - val_accuracy: 0.9618 - 4s/epoch - 7ms/step
Epoch 2/5
540/540 - 2s - loss: 0.0973 - accuracy: 0.9697 - val_loss: 0.0760 - val_accuracy: 0.9765 - 2s/epoch - 4ms/step
Epoch 3/5
540/540 - 3s - loss: 0.0638 - accuracy: 0.9796 - val_loss: 0.0666 - val_accuracy: 0.9795 - 3s/epoch - 6ms/step
Epoch 4/5


In [ ]:
test_loss, test_accuracy = model.evaluate(test_data)

In [ ]:
print("Test Loss : {0:.2f}. Test accuracy: {1:.2f}%".format(test_loss, test_accuracy *100))